In [49]:
import glob
import pandas as pd
import numpy as np
from collections import Counter

In [50]:
#below choose which directory to work with (all files or test files)

path = 'homepage_csvs/test.myfreecams/' #change platform name after full stop to choose which test files to use
#path = 'homepage_csvs/chaturbate/'
#path = 'homepage_csvs/myfreecams/'
#path = 'homepage_csvs/bongacams/'
#path = 'homepage_csvs/livejasmin/'
#path = 'homepage_csvs/streamate/'
#path = 'homepage_csvs/ebonycams/'

#DEPENDING ON THE PLATFORM, MAKE SURE TO ALSO DEAL WITH FILE NAMES OF SAVED CSV LATER!

# get all filenames from the directory
csv_files = glob.glob(path + '*.csv')

## Collecting needed metrics from csv files
- this is code for reading multiple frontpage files and calculating metrics per timestamp
- code is based on the example from "basic_file_read" by Bernhard
- we collect metrics: for each day in the studied timeframe and also for each hour in 24hrs, not separating per date


In [ ]:

daily_dict = {} #create an empty dictionary to store all metrics for daily calculations
daily_tags = {} #create an empty dictionary to store a list of all hashtags per day
hourly_dict = {} #make the master dictionary for collecting hourly data
#IF HASHTAGS ARE INTERESTING PER TIME IN DAY, THEN HERE THAT SHOULD BE ADDED THE SAME WAY AS IN METRICS PER DAY

counter = 1

# iterate over all files to create an array of lists with all metrics for each day
for filename in sorted(csv_files):

    # extract date from filename
    #print(filename)
    fileparts = filename.split('_')
    smaller_parts = fileparts[1].split('/')
    platform_split = smaller_parts[2].split('2')
    platform = platform_split[0]
    #print(platform)
    
    # myfreecams has no underscore after platform name, so the date/time extraction is different:
    if platform == "myfreecams":
        myfreecams_split = smaller_parts[2].split('myfreecams')
        date = myfreecams_split[1]
        time = fileparts[2]
        #print(time)
        #print(date)
    else: #for the rest of the platforms it's simple:
        date = fileparts[2]
        time = fileparts[3]
        #print(time)
        #print(date)


    timeparts = time.split('-')
    hour = f"0000-00-00 {timeparts[0]}:{timeparts[1]}:00" #the weird format is because i need that for some visualizations
    #print(hour)
    
    print(f"{counter} platform:{platform} date:{date} time:{hour}")
    counter += 1

    if date not in daily_dict: #add the date to a dictionary with keys for each date
        daily_dict[date] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each date for metrics and list for each value
        daily_tags[date] = [] #daily also creating hashtags, but not by hour

    
    if hour not in hourly_dict: #add the hour to a dictionary with keys for each hour
        hourly_dict[hour] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each halfhour for metrics and list for each value


    # __________________________read CSV file and get metrics________________________________________

    df = pd.read_csv(filename) #open the csv as dataframe
    files = 1
    
    all_performers_count = len(df)
    
    if "offline" in df.columns:
        offline_count = df.offline.sum()
        df = df[df["offline"] == False] #filtering out the rows that have offline performers
        #print("offline performers might exist (ebony), but were removed")
    elif "away" in df.columns:
        offline_count = df.away.sum()
        df = df[df["away"] == False] #filtering out the rows that have offline performers
        #print("offline performers might existe (bonga), but were removed")
    else:
        offline_count = 0
        #print("this plaform does not save offline performers, proceeding >>")
    
    
    perf_count = len(df) #number of shows in each csv file
    print(f"number of online performers:{perf_count}, number of offline performers: {offline_count}, number of all performers: {all_performers_count}")
    print(f"{perf_count}?")
    print(all_performers_count - offline_count) #check if i did not fuck up by checking if the numbers are the same
    
# _______________viewers___________

    if "viewers" in df.columns:
        df['viewers'] = df['viewers'].astype(str) #first make all viewer counts into string
        df['viewers'] = df['viewers'].str.replace('+', '') #then remove any "+"" from strings
        df["viewers"] = pd.to_numeric(df["viewers"]) #then make them into integers again -- this is to fix bongacams issue of having "99999+" as a number of viewers
        #df["viewers"] = df["viewers"].astype(int)
        view_sum = df["viewers"].sum() #sum of viewers in each csv file
        #print(f"got sum of viewers chaturbate/bongacams: {view_sum}")
    elif "room_count" in df.columns:
        view_sum = df["room_count"].sum()
        #print(f"got sum of viewers myfreecams: {view_sum}")    
    else:
        view_sum = 0
        #print("no viewer numbers for this platform")
    
    #try:
    #    show_len_sum = sum(df["time"]) #sum of show time in each csv file
    #except:
    #    show_len_sum = 0
    #    print("this platform does not provide show lengths")
    #skipping this part because it is only on chaturbate and is not accurate, because it is not actually show length, 
    #but how long the performer has been online at the moment of scrape
    
# _______________gender, couples___________
    #this is only for chaturbate:
    if "female" in df.columns:    #they all go together
        female_count = df.female.sum() #number of female performers in chaturbate files
        male_count = df.male.sum() #number of male performers
        trans_count = df.trans.sum() #number of trans performers
        couple_count = df.couple.sum() #number of couples performing

    else:
        female_count = 0
        male_count = 0
        trans_count = 0
        couple_count = 0
        #print("this platform does not provide gender indications")

# _______________vibrator use___________
    if "vibrator" in df.columns:
        vibrator_count = df.vibrator.sum() #number of performances with smart vibrators activated 
    else:
        vibrator_count = 0
        #print("this platform does not provide vibrator indication")

# _______________new performers___________
    if "new" in df.columns: 
        new_count = df.new.sum() #number of new performers 
    else:
        new_count = 0
        #print("this platform does not provide new performer indication")
    
# _______________performers in private shows___________   
    if "private" in df.columns:
        private_count = df.private.sum()  #number of performers in a private show at the moment (bongacams)
        #print(f"got bongacams privates: {private_count}")
    elif "private_show" in df.columns:
        private_count = df.private_show.sum() + df.true_private_show.sum() #number of performers in a private show at the moment (myfreecams)
        #print(f"got myfreecams privates: {private_count}")
    else:
        private_count = 0
        #print("this platform does not provide private show indication")
    
# _______________promoted performers___________    

    if "promoted" in df.columns:
        promoted_count = df.promoted.sum()
        #print(f"got promoted performers (livejasmin/chaturbate): {promoted_count}")
    elif "lifted_up_webcam_model" in df.columns:
        promoted_count = df.lifted_up_webcam_model.sum()
        #print(f"got promoted performers (bongacams): {promoted_count}")
    else:
        promoted_count = 0
        #print("this platform does not provide promoted performer indication")

# _______________performers in group shows___________   
    if "group" in df.columns:
        group_count = df.group.sum() #bonga
        #print(f"got performers in group shows (bongacams): {group_count}")
    elif "group_show" in df.columns:
        group_count = df.group_show.sum() #myfreecams
        #print(f"got performers in group shows (myfreecams): {group_count}")
    else:
        group_count = 0
        #print("this platform does not provide group show indication")


   # __________________________put metrics in a dictionary________________________________________
    # add all the new collected values in the nested dictionary for each day
    daily_dict[date]["perf_counts"].append(perf_count)
    daily_dict[date]["view_sums"].append(view_sum)
    daily_dict[date]["female_counts"].append(female_count)
    daily_dict[date]["male_counts"].append(male_count)
    daily_dict[date]["trans_counts"].append(trans_count)
    daily_dict[date]["couple_counts"].append(couple_count)
    daily_dict[date]["vibrator_counts"].append(vibrator_count)
    daily_dict[date]["new_counts"].append(new_count)
    daily_dict[date]["private_counts"].append(private_count)
    daily_dict[date]["offline_counts"].append(offline_count)
    daily_dict[date]["promoted_counts"].append(promoted_count)
    daily_dict[date]["group_counts"].append(group_count)
    daily_dict[date]["number_files"].append(files)
    print("collected daily metrics")

    # add all the new collected values in the nested dictionary for each hour in 24hrs
    hourly_dict[hour]["perf_counts"].append(perf_count)
    hourly_dict[hour]["view_sums"].append(view_sum)
    hourly_dict[hour]["female_counts"].append(female_count)
    hourly_dict[hour]["male_counts"].append(male_count)
    hourly_dict[hour]["trans_counts"].append(trans_count)
    hourly_dict[hour]["couple_counts"].append(couple_count)
    hourly_dict[hour]["vibrator_counts"].append(vibrator_count)
    hourly_dict[hour]["new_counts"].append(new_count)
    hourly_dict[hour]["private_counts"].append(private_count)
    hourly_dict[hour]["offline_counts"].append(offline_count)
    hourly_dict[hour]["promoted_counts"].append(promoted_count)
    hourly_dict[hour]["group_counts"].append(group_count)
    hourly_dict[hour]["number_files"].append(files)
    print("collected hourly metrics")



# __________________________collect hashtags per day and put in dictionary________________________________________

    if "tags" in df.columns:
        hashtags_list = df["tags"].tolist() #makes a list of all the hashtags used that day
        
        hashtags = []
        for object in hashtags_list:
            #print(object)
            if type(object) is str:
                #print(type(object))
                object = object.split(",")
                for word in object:
                    hashtags.append(word)
            else:
                #print(type(object))
                hashtags.append(object)
        #print(hashtags)

    #add the collected hashtags in their own dictionary
        daily_tags[date]+=hashtags
        print("collected daily hashtags")
    else:
        hashtags_list = 0
        print("this platform does not provide hashtags")


#print(daily_dict)
#print(hourly_dict)
#print(daily_tags)

## Making daily metrics output file with calculations

In [ ]:
daily_avgs = []

# next step is to iterate over array to calculate daily averages and counts

for date in daily_dict:  #for each date calculate:
    daily_files_sum = np.sum(daily_dict[date]["number_files"]) #how many files collected for that day
    daily_shows_avg = np.mean(daily_dict[date]["perf_counts"]) #average length of page (number of performances - the ones that are online) (all platforms)
    daily_offline_avg = np.mean(daily_dict[date]["offline_counts"]) #average of how many people on the list were offline (bongacams and myfreecams)
    daily_views_sum = sum(daily_dict[date]["view_sums"]) #sum of all viewers that day (chaturbate, bongacams and myfreecams)
    daily_views_sum_avg = daily_views_sum / daily_files_sum #added this now -- average of total viewers on the platform that day (chaturbate, bongacams and myfreecams)
    daily_views_avg = daily_views_sum / sum(daily_dict[date]["perf_counts"]) #average number of viewers in a show that day (chaturbate, bongacams and myfreecams)
    
    daily_female_avg = np.mean(daily_dict[date]["female_counts"]) #only chaturbate
    daily_female_percent = sum(daily_dict[date]["female_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_male_avg = np.mean(daily_dict[date]["male_counts"]) #only chaturbate
    daily_male_percent = sum(daily_dict[date]["male_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_trans_avg = np.mean(daily_dict[date]["trans_counts"]) #only chaturbate
    daily_trans_percent = sum(daily_dict[date]["trans_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    daily_couple_avg = np.mean(daily_dict[date]["couple_counts"]) #only chaturbate
    daily_couple_percent = sum(daily_dict[date]["couple_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #only chaturbate
    
    daily_vibrator_avg = np.mean(daily_dict[date]["vibrator_counts"]) #(bongacams and livejasmin)
    daily_vibrator_percent = sum(daily_dict[date]["vibrator_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #(bongacams and livejasmin)
    daily_new_avg = np.mean(daily_dict[date]["new_counts"]) #bongacams, chaturbate, myfreecams, livejasmin
    daily_new_percent = sum(daily_dict[date]["new_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams, chaturbate, myfreecams, livejasmin
    daily_promoted_avg = np.mean(daily_dict[date]["promoted_counts"]) #bongacams, livejasmin, chaturbate
    daily_promoted_percent = sum(daily_dict[date]["promoted_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams, livejasmin, chaturbate

    daily_private_avg = np.mean(daily_dict[date]["private_counts"]) #bongacams and myfreecams
    daily_private_percent = sum(daily_dict[date]["private_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams and myfreecams
    daily_group_avg = np.mean(daily_dict[date]["group_counts"]) #bongacams and myfreecams
    daily_group_percent = sum(daily_dict[date]["group_counts"]) / sum(daily_dict[date]["perf_counts"]) * 100 #bongacams and myfreecams


    #then add everything to a nice dictionary
    daily_avgs.append({'date':date, 'number_files': daily_files_sum, 'shows_average':round(daily_shows_avg, 2), 'offline_average':round(daily_offline_avg, 2), 
    'viewers_sum':daily_views_sum , 'viewers_sum_avg':round(daily_views_sum_avg, 2), 'viewers_average_in_show':round(daily_views_avg, 2),
    'average_vibrator':round(daily_vibrator_avg,2), 'percentage_vibrator':round(daily_vibrator_percent,2), 'average_new':round(daily_new_avg,2), 'percentage_new':round(daily_new_percent,2),
    'average_promoted':round(daily_promoted_avg,2), 'percentage_promoted':round(daily_promoted_percent,2), 'average_private':round(daily_private_avg,2), 'percentage_private':round(daily_private_percent,2),
    'average_group':round(daily_group_avg,2), 'percentage_group':round(daily_group_percent,2), 'average_females':round(daily_female_avg,2), 'percentage_females':round(daily_female_percent,2), 
    'average_males':round(daily_male_avg,2), 'percentage_males':round(daily_male_percent,2), 'average_trans':round(daily_trans_avg,2), 'percentage_trans':round(daily_trans_percent,2), 
    'average_couples':round(daily_couple_avg), 'percentage_couples':round(daily_couple_percent,2)})
    #print(daily_avgs)

#and make a dataframe from the dictionary
df = pd.DataFrame.from_dict(daily_avgs)
#df.to_csv('daily_metrics_myfreecams.csv',index=False) #and save it as a csv file
df

In [54]:
# here i make a file with the counted hashtags per day
# FUTURE: not include all hashtags, but only those that are counted above a threshold?

daily_tags_counted = []

for date in daily_tags:  #iterating though every date
    daily_counts = Counter(daily_tags[date]) #instead of having a list of all hashtags, this makes a dictionary with counts of each hashtag
    #print(daily_counts)

    for tag in daily_counts: 
        daily_tags_counted.append({"date": date, "hashtag": tag , "count": daily_counts[tag]})
        #daily_tags_counted[date][tag] = daily_counts[tag]

df = pd.DataFrame.from_dict(daily_tags_counted)
#df.to_csv('daily_hashtags_myfreecams.csv',index=False)
df
    

""


## Making halfhourly metrics output file with calculations

In [ ]:
hourly_avgs = []

# next step is to iterate over array to calculate hourly metrics
for hour in hourly_dict:  #for each hour in day calculate:
    hourly_files_sum = np.sum(hourly_dict[hour]["number_files"]) #how many files collected for that halfhour interval
    hourly_shows_avg = np.mean(hourly_dict[hour]["perf_counts"]) #average length of page (number of performances - the ones that are online) (all platforms)
    hourly_offline_avg = np.mean(hourly_dict[hour]["offline_counts"]) #average of how many people on the list were offline (bongacams and myfreecams)
    hourly_views_sum = sum(hourly_dict[hour]["view_sums"]) #sum of all viewers on the platform over all collected data for that halfhour period (chaturbate, bongacams and myfreecams)
    hourly_views_sum_avg = hourly_views_sum / hourly_files_sum #average number of viewers on the platform (chaturbate, bongacams and myfreecams)
    hourly_views_avg =  hourly_views_sum / sum(hourly_dict[hour]["perf_counts"]) #average number of viewers in a show per halfhour (chaturbate, bongacams and myfreecams)

    hourly_female_avg = np.mean(hourly_dict[hour]["female_counts"]) #only chaturbate
    hourly_female_percent = sum(hourly_dict[hour]["female_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_male_avg = np.mean(hourly_dict[hour]["male_counts"]) #only chaturbate
    hourly_male_percent = sum(hourly_dict[hour]["male_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_trans_avg = np.mean(hourly_dict[hour]["trans_counts"]) #only chaturbate
    hourly_trans_percent = sum(hourly_dict[hour]["trans_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    hourly_couple_avg = np.mean(hourly_dict[hour]["couple_counts"]) #only chaturbate
    hourly_couple_percent = sum(hourly_dict[hour]["couple_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #only chaturbate
    
    hourly_vibrator_avg = np.mean(hourly_dict[hour]["vibrator_counts"]) #(bongacams and livejasmin)
    hourly_vibrator_percent = sum(hourly_dict[hour]["vibrator_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #(bongacams and livejasmin)
    hourly_new_avg = np.mean(hourly_dict[hour]["new_counts"]) #bongacams, chaturbate, myfreecams, livejasmin
    hourly_new_percent = sum(hourly_dict[hour]["new_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams, chaturbate, myfreecams, livejasmin
    hourly_promoted_avg = np.mean(hourly_dict[hour]["promoted_counts"]) #bongacams, livejasmin, chaturbate
    hourly_promoted_percent = sum(hourly_dict[hour]["promoted_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams, livejasmin, chaturbate

    hourly_private_avg = np.mean(hourly_dict[hour]["private_counts"]) #bongacams and myfreecams
    hourly_private_percent = sum(hourly_dict[hour]["private_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams and myfreecams
    hourly_group_avg = np.mean(hourly_dict[hour]["group_counts"]) #bongacams and myfreecams
    hourly_group_percent = sum(hourly_dict[hour]["group_counts"]) / sum(hourly_dict[hour]["perf_counts"]) * 100 #bongacams and myfreecams


    #then add everything to a nice dictionary
    hourly_avgs.append({'hour':hour, 'number_files': hourly_files_sum, 'shows_average':round(hourly_shows_avg, 2), 'offline_average':round(hourly_offline_avg, 2),
    'viewers_sum':hourly_views_sum, 'viewers_sum_avg':round(hourly_views_sum_avg,2), 'viewers_average':round(hourly_views_avg, 2),
    'average_vibrator':round(hourly_vibrator_avg,2), 'percentage_vibrator':round(hourly_vibrator_percent,2), 'average_new':round(hourly_new_avg,2), 'percentage_new':round(hourly_new_percent,2),
    'average_promoted':round(hourly_promoted_avg,2), 'percentage_promoted':round(hourly_promoted_percent,2), 'average_private':round(hourly_private_avg,2), 'percentage_private':round(hourly_private_percent,2),
    'average_group':round(hourly_group_avg,2), 'percentage_group':round(hourly_group_percent,2), 'average_females':round(hourly_female_avg,2), 'percentage_females':round(hourly_female_percent,2), 
    'average_males':round(hourly_male_avg,2), 'percentage_males':round(hourly_male_percent,2), 'average_trans':round(hourly_trans_avg,2), 'percentage_trans':round(hourly_trans_percent,2), 
    'average_couples':round(hourly_couple_avg), 'percentage_couples':round(hourly_couple_percent,2)})
    #print(hourly_avgs)

#and make a dataframe from the dictionary
df = pd.DataFrame.from_dict(hourly_avgs)
#df.to_csv('hourly_metrics_ebonycams.csv',index=False) #and save it as a csv file
df


## -- IGNORE THIS --
Old version 

Bellow: collecting metrics separately per half an hour

In [ ]:
#make the master dictionary for collecting hourly data
hourly_dict = {}
#IF HASHTAGS ARE INTERESTING PER TIME IN DAY, THEN HERE THAT SHOULD BE ADDED THE SAME WAY AS IN METRICS PER DAY
counter = 1


# iterate over all files to create an array of lists with all metrics for each hour in a day
for filename in sorted(csv_files):

    #print(filename)
    fileparts = filename.split('_')
    smaller_parts = fileparts[1].split('/')
    platform_split = smaller_parts[2].split('2')
    platform = platform_split[0]
    #print(platform)
    
    if platform == "myfreecams":
        time = fileparts[2]
        print(time)
    else:
        time = fileparts[3]
        print(time)
   
    timeparts = time.split('-')
    hour = f"0000-00-00 {timeparts[0]}:{timeparts[1]}:00"
    #print(hour)
    
    print(str(counter) + ' ' + hour + ' ' + platform)
    counter += 1


    if hour not in hourly_dict: #add the hour to a dictionary with keys for each hour
        hourly_dict[hour] = {"perf_counts":[], "view_sums":[], "show_length_sums":[], "female_counts":[], "male_counts":[], "trans_counts":[], "couple_counts":[], 
                            "vibrator_counts":[], "new_counts":[], "private_counts":[], "group_counts":[], "offline_counts":[], "promoted_counts":[], "number_files":[]} 
                            #create a dictionary within each date for metrics and list for each value


    # __________________________read CSV file and get metrics________________________________________
    # this obviously could be done together with hourly metrics, so the files don't need to be opened twice, but for now i am keeping it this way
    # because i already have run the metrics for days in the timeframe

    df = pd.read_csv(filename) #open the csv as dataframe
    files = 1
    
    all_performers_count = len(df)
    
    if "offline" in df.columns:
        offline_count = df.offline.sum()
        df = df[df["offline"] == False] #filtering out the rows that have offline performers
        #print("offline performers might exist (ebony), but were removed")
    elif "away" in df.columns:
        offline_count = df.away.sum()
        df = df[df["away"] == False] #filtering out the rows that have offline performers
        #print("offline performers might existe (bonga), but were removed")
    else:
        offline_count = 0
        #print("this plaform does not save offline performers, proceeding >>")
    
    
    perf_count = len(df) #number of shows in each csv file
    print(f"number of online performers:{perf_count}, number of offline performers: {offline_count}, number of all performers: {all_performers_count}")
    print(f"{perf_count}?")
    print(all_performers_count - offline_count)
    
# _______________viewers___________

    if "viewers" in df.columns:
        df['viewers'] = df['viewers'].astype(str) #first make all viewer counts into string
        df['viewers'] = df['viewers'].str.replace('+', '') #then remove any "+"" from strings
        df["viewers"] = pd.to_numeric(df["viewers"]) #then make them into integers again -- this is to fix bongacams issue of having "99999+" as a number of viewers
        #df["viewers"] = df["viewers"].astype(int)
        view_sum = df["viewers"].sum() #sum of viewers in each csv file
        print(f"got sum of viewers chaturbate/bongacams: {view_sum}")
    elif "room_count" in df.columns:
        view_sum = df["room_count"].sum()
        print(f"got sum of viewers myfreecams: {view_sum}")    
    else:
        view_sum = 0
        print("no viewer numbers for this platform")
    
    #try:
    #    show_len_sum = sum(df["time"]) #sum of show time in each csv file
    #except:
    #    show_len_sum = 0
    #    print("this platform does not provide show lengths")
    #skipping this part because it is only on chaturbate and is not accurate, because it is not actually show length, 
    #but how long the performer has been online at the moment of scrape
    
# _______________gender, couples___________
    #this is only for chaturbate:
    if "female" in df.columns:    #they all go together
        female_count = df.female.sum() #number of female performers in chaturbate files
        male_count = df.male.sum() #number of male performers
        trans_count = df.trans.sum() #number of trans performers
        couple_count = df.couple.sum() #number of couples performing

    else:
        female_count = 0
        male_count = 0
        trans_count = 0
        couple_count = 0
        #print("this platform does not provide gender indications")

# _______________vibrator use___________
    if "vibrator" in df.columns:
        vibrator_count = df.vibrator.sum() #number of performances with smart vibrators activated 
    else:
        vibrator_count = 0
        #print("this platform does not provide vibrator indication")

# _______________new performers___________
    if "new" in df.columns: 
        new_count = df.new.sum() #number of new performers 
    else:
        new_count = 0
        #print("this platform does not provide new performer indication")
    
# _______________performers in private shows___________   
    if "private" in df.columns:
        private_count = df.private.sum()  #number of performers in a private show at the moment (bongacams)
        #print(f"got bongacams privates: {private_count}")
    elif "private_show" in df.columns:
        private_count = df.private_show.sum() + df.true_private_show.sum() #number of performers in a private show at the moment (myfreecams)
        #print(f"got myfreecams privates: {private_count}")
    else:
        private_count = 0
        #print("this platform does not provide private show indication")
    
# _______________promoted performers___________    

    if "promoted" in df.columns:
        promoted_count = df.promoted.sum()
        #print(f"got promoted performers (livejasmin/chaturbate): {promoted_count}")
    elif "lifted_up_webcam_model" in df.columns:
        promoted_count = df.lifted_up_webcam_model.sum()
        #print(f"got promoted performers (bongacams): {promoted_count}")
    else:
        promoted_count = 0
        #print("this platform does not provide promoted performer indication")

# _______________performers in group shows___________   
    if "group" in df.columns:
        group_count = df.group.sum() #bonga
        #print(f"got performers in group shows (bongacams): {group_count}")
    elif "group_show" in df.columns:
        group_count = df.group_show.sum() #myfreecams
        #print(f"got performers in group shows (myfreecams): {group_count}")
    else:
        group_count = 0
        #print("this platform does not provide group show indication")


   # __________________________put metrics in a dictionary________________________________________
    # add all the new collected values in the nested dictionary
    hourly_dict[hour]["perf_counts"].append(perf_count)
    hourly_dict[hour]["view_sums"].append(view_sum)
    hourly_dict[hour]["female_counts"].append(female_count)
    hourly_dict[hour]["male_counts"].append(male_count)
    hourly_dict[hour]["trans_counts"].append(trans_count)
    hourly_dict[hour]["couple_counts"].append(couple_count)
    hourly_dict[hour]["vibrator_counts"].append(vibrator_count)
    hourly_dict[hour]["new_counts"].append(new_count)
    hourly_dict[hour]["private_counts"].append(private_count)
    hourly_dict[hour]["offline_counts"].append(offline_count)
    hourly_dict[hour]["promoted_counts"].append(promoted_count)
    hourly_dict[hour]["group_counts"].append(group_count)
    hourly_dict[hour]["number_files"].append(files)
